# Attributs de type liste

* Auteurs/trices : **TODO**

Ce chapitre traite des attributs de type liste et des différents types de requêtes que l'on peut vouloir faire sur de tels attributs

Le fichier que vous devez modifier pour ce chapitre est `mongo_book/content/02_listes.md`.